In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [6]:
x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')

f = x*x*y + y + 2

In [7]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [8]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [9]:
sess.close()

In [10]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [11]:
print(result)

42


In [12]:
init = tf.global_variables_initializer() # prepare all init node

with tf.Session() as sess:
    init.run() # actually initializes all the variables
    result = f.eval()

In [13]:
# Interactive session sets itself as the default session once created
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [14]:
# Managing graphs
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [15]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [16]:
x2.graph is graph 

True

In [17]:
x2.graph is tf.get_default_graph()

False

In [18]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval()) # 10
    print(z.eval()) # 15

10
15


In [19]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape


x = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = 'x')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = 'y')
XT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, x)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
traning_op = tf.assign(theta, theta- learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
    sess.run(training_op)
    
best_theta = theta.eval()



In [ ]:
# using an optimizer
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
    sess.run(training_op)
    
best_theta = theta.eval()



In [ ]:
# using different types of optimizers
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = 'y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)
# the momentum optimizer converges much faster than gradient descent


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE = ', mse.eval())
    sess.run(training_op)
    
best_theta = theta.eval()



In [21]:
# feeding data to the training algorithm
# trying to implement Mini-batch gradient descent

A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5
with tf.Session() as sess:
    b_val_1 = B.eval(feed_dict = {A:[[1,2,3]]})
    b_val_2 = B.eval(feed_dict = {A:[[4,5,6],[7,8,9]]})
print(b_val_1)
print(b_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [ ]:
#using the momentum optimizer
# also saving and restoring the model

X = tf.placeholder(tf.float32, shape = (None, n+1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    return x_batch, y_batch


theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            save_path = saver.save(sess, '/tmp/momentum_model.ckpt')
            sess.run(training_op, feed_dict = [X:x_batch, y:y_batch])
            
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/momentum_model_final.ckpt')

In [ ]:
#using the Gradient descent optimizer
# also saving and restoring the model
X = tf.placeholder(tf.float32, shape = (None, n+1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    return x_batch, y_batch


theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            save_path = saver.save(sess, '/tmp/gradient_descent_opti.ckpt')
            sess.run(training_op, feed_dict = [X:x_batch, y:y_batch])
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/gradient_descent_opti_final.ckpt')

In [ ]:
# restoring models


# restore momentum_optimizer
with tf.Session() as sess:
    saver.restore(sess, '/tmp/momentum_model_final.ckpt')
    
    
    
# restore gradient descent optimizer
with tf.Session() as sess:
    saver.restore(sess, '/tmp/gradient_descent_opti_final.ckpt')
    
    
# specifying values to save or restor
saver = tf.train.Saver({'weights': theta})

# loading the graph structure of the model saved in the .meta file using
tf.train.import_meta_graph()

saver = tf.train.import_meta_graph('/tmp/gradient_descent_opti_final.ckpt.meta')
with tf.Session() as sess:
    saver.restore(sess,'/tmp/gradient_descent_opti_final.ckpt')
# this allows to fully save and restore model, including both the graph 
# structure and the variable values, without having to search for the code that built it


In [ ]:
# visualizing graph and training curves using tensorboard
# adding this to the beginning of the code

from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)
mse_summary = tf.summary.scaler('MSE', mse)
file_writer= tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
# updating  the execution phase to evaluate the mse_summary on the using the gradient descent optimizer

X = tf.placeholder(tf.float32, shape = (None, n+1), name = 'X')
y = tf.placeholder(tf.float32, shape = (None, 1), name = 'y')

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    return x_batch, y_batch


theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict = {X:x_batch, y:y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
                save_path = saver.save(sess, '/tmp/gradient_descent_opti.ckpt')
            sess.run(training_op, feed_dict = [X:x_batch, y:y_batch])
    best_theta = theta.eval()
    save_path = saver.save(sess, '/tmp/gradient_descent_opti_final.ckpt')
    file_writer.close()

In [ ]:
# When dealing with more complex models such as neural networks, the graph can 
# easily become cluttered with thousands of nodes. To avoid this, you can crate name
# scope to group related nodes.

#for example, defining the error and mse ops within a name scope called 'loss'
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = 'mse')
    
    print(error.op.name)
    print(mse.op.name)

In [ ]:
# Modularity
# rectified linear units(RELU).
# adding the output of two rectified linear units

n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')

w1 = tf.Variable(tf.random_normal((n_features, 1)),name = 'weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name = 'weights2')
b1 = tf.Variable(0.0, name = 'bias1')
b2 = tf.Variable(0.0, name = 'bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name = 'z1')
z2 = tf.add(tf.matmul(X, w2), b2, name = 'z2')

relu1 = tf.maximum(z1, 0., name = 'relu1')
relu2 = tf.maximum(z1, 0., name = 'relu2')


output = tf.add(relu1, relu2, name = 'output')


def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name = 'weights')
    b = tf.Variable(0.0, name = 'bias')
    z = tf.add(tf.matmul(X,w), b, name = 'z')
    return tf.maximum(z, 0., name = 'relu')

n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = 'X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name = 'output')